<style type="text/css">.tg  {border-collapse:collapse;border-spacing:0;}.tg td{border-color:rgb(16, 137, 182);border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}.tg th{border-color:rgb(16, 137, 182);border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}.tg .tg-73oq{border-color:rgb(10, 89, 162);text-align:left;vertical-align:middle}.tg .tg-42lt{border-color:rgb(10, 89, 162);text-align:center;vertical-align:middle}.tg .tg-5qt9{font-size:small;text-align:left;vertical-align:middle}</style><table class="tg"><thead>  <tr>    <th class="tg-73oq"><img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="Argopy logo" width="120" height="60"></th>    <th class='tg-42lt'><h1>Vertical interpolation & Binning</h1></th>  </tr></thead><tbody>  <tr>    <td class="tg-5qt9" colspan="2"><span style="font-weight:bold">Author :</span> <a href="//annuaire.ifremer.fr/cv/22144" target="_blank" rel="noopener noreferrer">K. Balem</a></td>  </tr>  <tr>    <td class="tg-5qt9" colspan="2">🏷️ This notebook is compatible with Argopy versions &gt;= <a href="https://argopy.readthedocs.io/en/v1.3.1" target="_blank" rel="noopener noreferrer">v1.3.1</a></td>  </tr>  <tr>    <td class="tg-5qt9" colspan="2">© <a href="https://github.com/euroargodev/argopy-training/blob/main/LICENSE" target="_blank" rel="noopener noreferrer">European Union Public Licence (EUPL) v1.2</a>, see at the bottom of this notebook for more.</td>  </tr></tbody></table>
**Description:**

This notebook describes:
- how to transform Argo data from a set of points to a collection of profiles,
- how to interpolate Argo data onto standard depth levels,
- how to bin vertically Argo data.

This notebook is based on the [Argopy documentation (↗)](https://argopy.readthedocs.io/en/v1.3.1/user-guide/working-with-argo-data/data_manipulation.html#) where you can find more details on each function.


**Table of Contents**
- [Points vs Profiles](#points-vs-profiles)
    - [✏️ EXERCISE](#✏️-exercise)
- [Vertical interpolation](#vertical-interpolation)
    - [🛟 NOTE](#🛟-note)
    - [✏️ EXERCISE](#✏️-exercise)
- [Pressure levels: Group-by bins](#pressure-levels:-group-by-bins)
    - [🔍 Pro tip](#🔍-pro-tip)
    - [✏️ EXERCISE](#✏️-exercise)
***

Let's start with the usual import:

In [ ]:
from argopy import DataFetcher

And to prevent cell output to be too large, we won't display xarray object attributes:

In [ ]:
import xarray as xr
xr.set_options(display_expand_attrs = False)

## Points vs Profiles
By default, fetched data are returned as a 1D array collection of measurements, ie a set of points.

In [ ]:
f = DataFetcher().region([-75, -55, 30., 40., 0, 100., '2011-01-01', '2011-01-15'])
f 

<br>

The returned xarray dataset will only have one dimension : **N_POINTS**

In [ ]:
ds_points = f.data
ds_points

<br>

If you prefer to work with 2D collection of vertical profiles, simply transform the dataset with [Dataset.argo.point2profile() (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/xarray.Dataset.argo.point2profile.html#xarray.Dataset.argo.point2profile)  

This will return a transformed dataset where the **N_POINTS** dimension is replaced by **N_PROF** and **N_LEVELS**:

In [ ]:
ds_profiles = ds_points.argo.point2profile()
ds_profiles

You can simply reverse this transformation with the [Dataset.argo.profile2point() (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/xarray.Dataset.argo.profile2point.html#xarray.Dataset.argo.profile2point):

In [ ]:
ds = ds_profiles.argo.profile2point()
ds

#### ✏️ EXERCISE
Fetch data from floats 6904183 and 6903071 and transform the returned dataset into profiles.

In [ ]:
# Your code here

## Vertical interpolation
Once your dataset is a collection of vertical profiles, you can interpolate variables on standard pressure levels using [Dataset.argo.interp_std_levels() (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/xarray.Dataset.argo.interp_std_levels.html#xarray.Dataset.argo.interp_std_levels) with your levels as input.  
This interpolated dataset has a new vertical dimension : **PRES_INTERPOLATED**

Let's load some data to work with:

In [ ]:
f = DataFetcher().region([-75., -55., 30., 40., 0, 100., '2011-01-01', '2011-01-15'])
ds_profiles = f.data.argo.point2profile()
ds_profiles

Let's define our standard pressure levels down to 100db:

In [ ]:
import numpy as np
z_levels = np.arange(0., 100., 10)
z_levels

We can now interpolate all of our profiles onto those levels

In [ ]:
ds_interp = ds_profiles.argo.interp_std_levels(z_levels)
ds_interp

#### 🛟 NOTE

About the linear interpolation process :
- Only profiles that have a maximum pressure higher than the highest standard level are selected for interpolation.
- Remaining profiles must have at least five data points to allow interpolation.
- For each profile, the shallowest data point is repeated to the surface to allow a 0 standard level while avoiding extrapolation.

In our case, one profile is missing in the interpolated dataset, compare to the orginal dataset. 

This is because its maximum pressure does not reach one or more standard levels:

In [ ]:
ds_profiles['PRES'].max('N_LEVELS').values

#### ✏️ EXERCISE
Fetch the data from float 1900547, transform the dataset into profiles, and interpolate onto standard pressure levels.

In [ ]:
# Your code here

In [ ]:
dsp['PRES'].max('N_LEVELS').values <= np.max(z_levels)

## Pressure levels: Group-by bins

If you prefer to avoid interpolation, you can opt for a pressure bins grouping reduction using [Dataset.argo.groupby_pressure_bins() (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/xarray.Dataset.argo.groupby_pressure_bins.html#xarray.Dataset.argo.groupby_pressure_bins).  

This method can be used to subsample and align an irregular dataset (pressure not being similar in all profiles) on a set of pressure bins. 

The output dataset could then be used to perform statistics along the N_PROF dimension because N_LEVELS will corresponds to similar pressure bins.

To illustrate this method, let’s start by fetching some data from a low vertical resolution float:

In [ ]:
f = DataFetcher(src='erddap', mode='expert').float(2901623)  # Low res float
ds = f.data
ds

Let’s now sub-sample these measurements along 250db bins, selecting values from the deepest pressure levels for each bins:

In [ ]:
bins = np.arange(0.0, np.max(ds["PRES"]), 250.0)
bins

In [ ]:
ds_binned = ds.argo.groupby_pressure_bins(bins=bins, select='deep')
ds_binned

See the new STD_PRES_BINS variable that hold the pressure bins definition.

The figure below shows the sub-sampling effect:

In [ ]:
import matplotlib.pyplot as plt
from argopy.plot import scatter_plot

# LETS PLOT FULL DATASET 
fig, ax, _, _ = scatter_plot(ds, 'PSAL', this_x='CYCLE_NUMBER', cmap='gray', cbar=True)

# LETS PLOT BINNED DATASET POINTS
plt.plot(ds_binned['CYCLE_NUMBER'], ds_binned['PRES'], 'r+')

# LETS PLOT OUR BINS LEVELS
plt.hlines(bins, ds['CYCLE_NUMBER'].min(), ds['CYCLE_NUMBER'].max(), color='k')
plt.hlines(ds_binned['STD_PRES_BINS'], ds_binned['CYCLE_NUMBER'].min(), ds_binned['CYCLE_NUMBER'].max(), color='r')

plt.title(ds.attrs['Fetched_constraints'])

<br>

The bin limits are shown with horizontal red lines, the original data are in the background colored scatter and the group-by pressure bins values are highlighted in red marks.

#### 🔍 Pro tip

The `select` option can take many different values, corresponding to different ways to process your data within the bin.

Check the full documentation of [Dataset.argo.groupby_pressure_bins() (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/xarray.Dataset.argo.groupby_pressure_bins.html#xarray.Dataset.argo.groupby_pressure_bins) for all the details. 

#### ✏️ EXERCISE
Fetch the data from float 5906993, and bin data using another select option. Illustrate the result.

In [ ]:
# Your code here


## 🏁 End of the notebook

***
#### 👀 Useful argopy commands
```python
argopy.reset_options()
argopy.show_options()
argopy.status()
argopy.clear_cache()
argopy.show_versions()
```
#### ⚖️ License Information
This Jupyter Notebook is licensed under the **European Union Public Licence (EUPL) v1.2**.

| Permissions      | Limitations     | Conditions                     |
|------------------|-----------------|--------------------------------|
| ✔ Commercial use | ❌ Liability     | ⓘ License and copyright notice |
| ✔ Modification   | ❌ Trademark use | ⓘ Disclose source              |
| ✔ Distribution   | ❌ Warranty      | ⓘ State changes                |
| ✔ Patent use     |                  | ⓘ Network use is distribution  |
| ✔ Private use    |                  | ⓘ Same license                 |

For more details, visit: [EUPL v1.2 Full Text (↗)](https://github.com/euroargodev/argopy-training/blob/main/LICENSE).

#### 🤝 Sponsor
![logo (↗)](https://raw.githubusercontent.com/euroargodev/argopy-training/refs/heads/main/for_nb_producers/disclaimer_argopy_EAONE.png)
***
